In [ ]:
###############   This code was built on pytorch lightning 2.2.5, pandas 2.2.0, etc. for neuralforecast compatibility   ##########
#######   Bike Sharing Data forecasting with MDN+N-HiTS   ########################

import sys, subprocess, importlib

def pipi(args):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + args)

# 1) Show the preloaded core versions; we will NOT replace them
import numpy as np
print("Using NumPy:", np.__version__)
try:
    import torch
    print("Using Torch:", torch.__version__)
except Exception:
    pass

# 2) Install neuralforecast WITHOUT pulling deps (so it won't alter numpy/torch)
pipi(["neuralforecast==1.7.4", "--no-deps"])

# 3) Install its python-only deps (don’t alter numpy/torch)
#    These versions work well with Colab’s current Torch & NumPy.
pipi([
    "pytorch-lightning==2.2.5",
    "torchmetrics==1.3.1",
    "einops>=0.7.0",
    "pandas>=2.2.0",        # just ensure present; will skip if already satisfied
    "scikit-learn>=1.3.0"   # for StandardScaler, train_test_split, etc.
])

# 4) (Optional) Verify imports now that everything is aligned
pipi([utilsforcast])pipi([coreforecast])import pandas as pd
from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS
from neuralforecast.losses.pytorch import PMM

print("Pandas:", pd.__version__)
print("NeuralForecast imported OK")

In [ ]:
# =========================
# COLAB PREP & PREPROCESSING (RUN FIRST)
# =========================
# - Loads your raw bike-sharing CSV
# - Normalizes to columns: ['unique_id','ds','y'] at HOURLY frequency
# - Writes a cleaned CSV for the modeling block

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# -------------------------
# CONFIG (EDIT HERE)
# -------------------------
CONFIG_PREP = {
    "CSV_PATH": "/content/drive/MyDrive/myproject/bike_raw.csv",
    "OUT_DIR": "/content/drive/MyDrive/myproject/results_mdn_nhits",
    "UNIQUE_ID": "series_0",   # use constant label or column name if it exists
    "FREQ": "h",
    "TIMESTAMP_COL": None,
    "DATE_COL": None,
    "HOUR_COL": None,
    "DEMAND_COL": None,
    "PARSE_DAYFIRST": False
}

# -------------------------
# Imports
# -------------------------
import os
import numpy as np
import pandas as pd

os.makedirs(CONFIG_PREP["OUT_DIR"], exist_ok=True)
CLEAN_PATH = os.path.join(CONFIG_PREP["OUT_DIR"], "bike_clean.csv")

# -------------------------
# Loader + hourly builder
# -------------------------
def _load_raw(path: str) -> pd.DataFrame:
    return pd.read_csv(path, low_memory=False)

def _build_hourly_frame(df: pd.DataFrame, cfg) -> pd.DataFrame:
    # demand column
    dem_col = cfg["DEMAND_COL"]
    if dem_col is None:
        for cand in ["y","cnt","demand"]:
            if cand in df.columns:
                dem_col = cand; break
    if dem_col is None:
        raise ValueError("No demand column found.")
    # timestamp
    if cfg["TIMESTAMP_COL"] and cfg["TIMESTAMP_COL"] in df.columns:
        ds = pd.to_datetime(df[cfg["TIMESTAMP_COL"]], dayfirst=cfg["PARSE_DAYFIRST"], errors="coerce")
    else:
        dcol = cfg["DATE_COL"] if cfg["DATE_COL"] in df.columns else None
        hcol = cfg["HOUR_COL"] if cfg["HOUR_COL"] in df.columns else None
        if dcol and hcol:
            day = pd.to_datetime(df[dcol], errors="coerce").dt.date
            hour = pd.to_numeric(df[hcol], errors="coerce").fillna(0).astype(int).clip(0,23)
            ds = pd.to_datetime(day.astype(str)) + pd.to_timedelta(hour, unit="h")
        else:
            for cand in ["ds","datetime","timestamp"]:
                if cand in df.columns:
                    ds = pd.to_datetime(df[cand], errors="coerce"); break
    if ds is None: raise ValueError("No usable timestamp found.")
    # unique_id
    uid = str(cfg["UNIQUE_ID"])
    if uid in df.columns: uid_vals = df[uid].astype(str)
    else: uid_vals = [uid]*len(df)
    # build
    out = pd.DataFrame({"unique_id": uid_vals, "ds": ds, "y": pd.to_numeric(df[dem_col], errors="coerce")})
    out = out.dropna(subset=["ds"]).groupby(["unique_id","ds"], as_index=False)["y"].sum()
    # fill hourly grid
    frames = []
    for u,g in out.groupby("unique_id"):
        g = g.sort_values("ds")
        idx = pd.date_range(g["ds"].min(), g["ds"].max(), freq=cfg["FREQ"])
        g = g.set_index("ds").reindex(idx)
        g["unique_id"]=u
        g["y"] = g["y"].interpolate(limit_direction="both").fillna(0)
        g = g.reset_index().rename(columns={"index":"ds"})
        frames.append(g)
    out = pd.concat(frames, ignore_index=True)
    return out[["unique_id","ds","y"]].sort_values(["unique_id","ds"]).reset_index(drop=True)

# -------------------------
# Run preprocessing
# -------------------------
raw = _load_raw(CONFIG_PREP["CSV_PATH"])
clean = _build_hourly_frame(raw, CONFIG_PREP)
clean.to_csv(CLEAN_PATH, index=False)
print(f"[PREP] Wrote cleaned dataset → {CLEAN_PATH}")
display(clean.head(3))

In [ ]:
# =========================
# MDN → N-HiTS (RUN SECOND)
# =========================
# - MDN is fit ONLY on TRAIN (Hour, weekday → y)
# - MDN features (total_mean, total_variance + mixtures) extrapolated to ALL timestamps
# - N-HiTS cross-validation with 5 replications, seeds 1..5
# - Saves:
#     * dataset_with_mdn_exog.csv
#     * cv_forecasts_all_seeds.csv (aligned test grid, ready for MAPE)
# =========================

import os, math, time, warnings
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from typing import Tuple
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# NeuralForecast
os.environ["NIXTLA_ID_AS_COL"] = "1"  # ensure 'unique_id' appears as a column in output
from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS
from neuralforecast.losses.pytorch import PMM, GMM, MAE as NF_MAE

# -------------------------
# HYPERPARAMETER CONFIG (EDIT HERE)
# -------------------------
CONFIG = {
    "CLEAN_CSV_PATH": "/content/drive/MyDrive/myproject/results_mdn_nhits/bike_clean.csv",
    "OUT_DIR": "/content/drive/MyDrive/myproject/results_mdn_nhits/",
    "FREQ": "h",                 # hourly

    # Partitioning (kept from your original logic)
    "VAL_SIZE": 2*7*24,          # 2 weeks of hours for validation during CV
    "TEST_SPLIT_Q": 0.80,        # last 20% timestamps are test (by ds quantile)

    # Forecast horizon & input window
    "HORIZON": 24 * 14,          # 2 weeks
    "INPUT_SIZE": 24 * 7,        # 1 week

    # MDN hyperparameters
    "MDN_MIXES": 10,
    "MDN_HIDDEN": 16,
    "MDN_EPOCHS": 20,
    "MDN_PATIENCE": 20,
    "MDN_LR": 1e-3,
    "MDN_LOG_EVERY": 25,         # print train/val loss every N epochs
    "MDN_VERBOSE": True,

    # N-HiTS hyperparameters
    "NHITS_LOSS": "PMM",         # "PMM" | "GMM" | "MAE"
    "NHITS_PMM_COMPONENTS": 15,  # used if PMM or GMM
    "NHITS_LR": 1e-4,
    "NHITS_STEPS": 1000,
    "NHITS_BATCH": 16,

    # Reproducibility & device
    "SEED": 42,
    "DEVICE": "cuda" if torch.cuda.is_available() else "cpu",  # prefer T4 if available

    # Replications
    "SEEDS_FOR_CV": [1, 2, 3, 4, 5],

    # Quick plot
    "MAKE_QUICK_PLOT": True
}

os.makedirs(CONFIG["OUT_DIR"], exist_ok=True)

# -------------------------
# Utilities & Seed
# -------------------------
def set_seed(seed: int = 42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(CONFIG["SEED"])
print(f"[INFO] Using device: {CONFIG['DEVICE']}")

# -------------------------
# MDN
# -------------------------
class MDN(nn.Module):
    """
    2-layer MDN for 1-D targets:
      inputs: features (e.g., Hour, weekday)
      outputs: mixture params mu[K], sigma[K], pi[K]
    """
    def __init__(self, input_dims: int, hidden_dims: int, n_mixes: int):
        super().__init__()
        self.n_mixes = n_mixes
        self.output_dims = 1
        self.h1 = nn.Linear(input_dims, hidden_dims)
        self.h2 = nn.Linear(hidden_dims, hidden_dims)
        self.out = nn.Linear(hidden_dims, n_mixes * self.output_dims + n_mixes + n_mixes)
        self.act = nn.ReLU()

    def forward(self, x: torch.Tensor):
        x = self.act(self.h1(x))
        x = self.act(self.h2(x))
        raw = self.out(x)
        return self._split_params(raw)

    def _split_params(self, raw: torch.Tensor):
        B = raw.size(0)
        K = self.n_mixes
        D = self.output_dims
        mu_end = K * D
        sig_end = mu_end + K
        mu = raw[:, :mu_end].view(B, K, D)
        sigma = raw[:, mu_end:sig_end].view(B, K, 1)
        pis = raw[:, sig_end:].view(B, K)
        sigma = torch.exp(torch.clamp(sigma, -10, 10)) + 1e-6
        pis = torch.softmax(pis, dim=1) + 1e-12
        return mu, sigma, pis

    @staticmethod
    def nll(mu: torch.Tensor, sigma: torch.Tensor, pis: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """Negative log-likelihood under Gaussian mixture."""
        B, K, _ = mu.shape
        y = target.view(B, 1, 1).expand(B, K, 1)
        dist = torch.distributions.Normal(loc=mu, scale=sigma)
        log_probs = dist.log_prob(y).sum(dim=2)
        weighted = log_probs + torch.log(pis)
        mx = weighted.max(dim=1, keepdim=True).values
        log_sum = mx + torch.log(torch.sum(torch.exp(weighted - mx), dim=1, keepdim=True))
        return -log_sum.mean()

    def mixture_mean_var(self, x: torch.Tensor):
        """Return (mean, var, mu[K], sigma[K], pis[K])."""
        mu, sigma, pis = self.forward(x)
        muK = mu.squeeze(-1)        # [B,K]
        sigK = sigma.squeeze(-1)    # [B,K]
        mean = torch.sum(pis * muK, dim=1)       # [B]
        var = torch.sum(pis * (sigK**2 + muK**2), dim=1) - mean**2
        return mean, var.clamp_min(1e-12), muK, sigK, pis

# -------------------------
# Data helpers
# -------------------------
def load_clean(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df["ds"] = pd.to_datetime(df["ds"])
    need = {"unique_id", "ds", "y"}
    if not need.issubset(df.columns):
        raise ValueError(f"Clean CSV must contain columns {need}")
    return df.sort_values(["unique_id", "ds"]).reset_index(drop=True)

def add_calendar(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["Hour"] = out["ds"].dt.hour
    out["weekday"] = out["ds"].dt.weekday
    return out

def _to_np(t: torch.Tensor) -> np.ndarray:
    """Safe tensor→NumPy conversion via Python list (no .numpy() bridge)."""
    return np.asarray(t.detach().cpu().tolist(), dtype=np.float32)

# -------------------------
# MDN training
# -------------------------
def train_mdn(
    features: np.ndarray,
    target: np.ndarray,
    n_mixes: int,
    hidden_dims: int,
    epochs: int,
    patience: int,
    lr: float,
    seed: int,
    device: str,
    log_every: int = 25,
    verbose: bool = True,
    out_dir: str = None
) -> Tuple[MDN, StandardScaler]:
    set_seed(seed)
    Xtr, Xva, ytr, yva = train_test_split(features, target, test_size=0.2, random_state=seed)

    # Scale target only
    ysc = StandardScaler()
    ytr_s = ysc.fit_transform(ytr.reshape(-1, 1))
    yva_s = ysc.transform(yva.reshape(-1, 1))

    Xtr_t = torch.tensor(Xtr, dtype=torch.float32, device=device)
    Xva_t = torch.tensor(Xva, dtype=torch.float32, device=device)
    ytr_t = torch.tensor(ytr_s, dtype=torch.float32, device=device)
    yva_t = torch.tensor(yva_s, dtype=torch.float32, device=device)

    model = MDN(input_dims=features.shape[1], hidden_dims=hidden_dims, n_mixes=n_mixes).to(device)
    opt = optim.Adam(model.parameters(), lr=lr)

    best = math.inf
    bad = 0
    best_state = None

    if verbose:
        print(f"[MDN] Start training (epochs={epochs}, patience={patience}, device={device})")

    t0 = time.time()
    for ep in range(1, epochs + 1):
        model.train()
        opt.zero_grad()
        mu, sig, pis = model(Xtr_t)
        loss = MDN.nll(mu, sig, pis, ytr_t)
        loss.backward()
        opt.step()

        model.eval()
        with torch.no_grad():
            mu_v, sig_v, pis_v = model(Xva_t)
            vloss = MDN.nll(mu_v, sig_v, pis_v, yva_t)

        if verbose and (ep == 1 or ep % log_every == 0 or ep == epochs):
            print(f"[MDN][{ep:04d}] train_nll={loss.item():.4f}  val_nll={vloss.item():.4f}")

        if vloss.item() < best - 1e-4:
            best = vloss.item()
            bad = 0
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
        else:
            bad += 1
            if bad >= patience:
                if verbose:
                    print(f"[MDN] Early stopping at epoch {ep}. Best val_nll={best:.4f}")
                break

    if best_state is not None:
        model.load_state_dict(best_state)
    if verbose:
        print(f"[MDN] Done in {time.time()-t0:.1f}s.")

    # quick mixture snapshot
    if verbose:
        with torch.no_grad():
            sample_in = torch.tensor(Xtr[:4], dtype=torch.float32, device=device)
            _, _, _, _, pi_s = model.mixture_mean_var(sample_in)
            print("[MDN] sample mixture π (first 4 rows):")
            print(_to_np(pi_s))

    # persist weights + scaler (optional)
    if out_dir:
        torch.save(model.state_dict(), os.path.join(out_dir, "mdn_state_dict.pt"))
        np.save(os.path.join(out_dir, "mdn_target_scaler_mean.npy"), ysc.mean_)
        np.save(os.path.join(out_dir, "mdn_target_scaler_scale.npy"), ysc.scale_)

    return model, ysc

def mdn_features_for_rows(model: MDN, y_scaler: StandardScaler, hours: np.ndarray, weekdays: np.ndarray, device: str) -> dict:
    """Return dict with arrays: total_mean, total_variance, mu_i, sigma_i, pi_i (original scale)."""
    X = np.column_stack([hours, weekdays]).astype(np.float32)
    Xt = torch.tensor(X, dtype=torch.float32, device=device)
    model.eval()
    with torch.no_grad():
        mean_s, var_s, muK, sigK, pis = model.mixture_mean_var(Xt)

    mean_s_np = _to_np(mean_s).reshape(-1, 1)
    var_s_np  = _to_np(var_s)
    mu_np     = _to_np(muK)   # [N, K] scaled
    sig_np    = _to_np(sigK)  # [N, K]
    pi_np     = _to_np(pis)   # [N, K]

    # back-transform
    total_mean = y_scaler.inverse_transform(mean_s_np).ravel()
    total_std  = (np.sqrt(var_s_np) * y_scaler.scale_[0]).ravel()
    total_var  = total_std ** 2

    mu_np  = mu_np  * y_scaler.scale_[0] + y_scaler.mean_[0]
    sig_np = sig_np * y_scaler.scale_[0]

    return {
        "total_mean": total_mean,
        "total_variance": total_var,
        "mu": mu_np,
        "sigma": sig_np,
        "pi": pi_np
    }

# -------------------------
# NHITS helpers
# -------------------------
def _make_loss(choice: str, comps: int):
    c = choice.upper()
    if c == "PMM": return PMM(n_components=comps, quantiles=[.5])
    if c == "GMM": return GMM(n_components=comps)
    if c == "MAE": return NF_MAE()
    raise ValueError("NHITS_LOSS must be one of {'PMM','GMM','MAE'}")

# -------------------------
# MAIN
# -------------------------
def main(cfg):
    device = cfg["DEVICE"]
    FREQ   = cfg["FREQ"]

    # 1) Load clean hourly series
    base = load_clean(cfg["CLEAN_CSV_PATH"])
    base = add_calendar(base)  # adds Hour, weekday

    # 2) Partition into train/val/test (threshold by ds)
    n_time = base["ds"].nunique()
    threshold_date = base["ds"].quantile(cfg["TEST_SPLIT_Q"])
    VAL_SIZE  = int(cfg["VAL_SIZE"])
    TEST_SIZE = base[base["ds"] > threshold_date]["ds"].nunique()

    # compute cutoffs
    unique_dates = sorted(base["ds"].unique())
    training_cutoff   = unique_dates[-(VAL_SIZE + TEST_SIZE)]
    validation_cutoff = unique_dates[-TEST_SIZE]

    # masks
    base["is_train"] = base["ds"] <= training_cutoff
    base["is_val"]   = (base["ds"] > training_cutoff) & (base["ds"] <= validation_cutoff)
    base["is_test"]  = base["ds"] > validation_cutoff

    # 3) Train MDN on TRAIN ONLY
    train_rows = base[base["is_train"]].copy()
    X_train = train_rows[["Hour","weekday"]].to_numpy(np.float32)
    y_train = train_rows["y"].to_numpy(np.float32)

    mdn, ysc = train_mdn(
        X_train, y_train,
        n_mixes=cfg["MDN_MIXES"],
        hidden_dims=cfg["MDN_HIDDEN"],
        epochs=cfg["MDN_EPOCHS"],
        patience=cfg["MDN_PATIENCE"],
        lr=cfg["MDN_LR"],
        seed=cfg["SEED"],
        device=device,
        log_every=cfg["MDN_LOG_EVERY"],
        verbose=cfg["MDN_VERBOSE"],
        out_dir=cfg["OUT_DIR"]
    )

    # 4) Extrapolate MDN features to ALL timestamps (train+val+test)
    feats_all = mdn_features_for_rows(
        mdn, ysc,
        hours=base["Hour"].to_numpy(),
        weekdays=base["weekday"].to_numpy(),
        device=device
    )
    base["total_mean"]     = feats_all["total_mean"]
    base["total_variance"] = feats_all["total_variance"]

    # (Optional) export mixture columns too
    K = cfg["MDN_MIXES"]
    for i in range(K):
        base[f"mu_{i+1}"]    = feats_all["mu"][:, i]
        base[f"sigma_{i+1}"] = feats_all["sigma"][:, i]
        base[f"pi_{i+1}"]    = feats_all["pi"][:, i]

    # Save enriched historical dataset
    enriched_path = os.path.join(cfg["OUT_DIR"], "dataset_with_mdn_exog.csv")
    base.to_csv(enriched_path, index=False)
    print(f"[MDN] Enriched dataset saved → {enriched_path}")

    # 5) Prepare NeuralForecast frame (must include futr exog columns)
    df_nf = base[["unique_id","ds","y","total_mean","total_variance","is_test"]].copy().sort_values(["unique_id","ds"])

    # 6) Build dense test grid we’ll fill with forecasts
    dense_test = df_nf[df_nf["is_test"]][["unique_id","ds","y"]].copy().reset_index(drop=True)

    # 7) Dense CV (step_size=1), five seeds, with MDN exog
    loss_obj = _make_loss(cfg["NHITS_LOSS"], cfg["NHITS_PMM_COMPONENTS"])
    H        = cfg["HORIZON"]
    INP      = cfg["INPUT_SIZE"]

    per_seed_frames = []
    for seed in cfg["SEEDS_FOR_CV"]:
        print(f"\n[CV] NHITS seed={seed}  (dense step_size=1)")
        model = NHITS(
            h=H,
            input_size=INP,
            loss=loss_obj,
            n_pool_kernel_size=[2,2,2],
            n_freq_downsample=[24,12,1],
            scaler_type='robust',
            max_steps=cfg["NHITS_STEPS"],
            learning_rate=cfg["NHITS_LR"],
            batch_size=cfg["NHITS_BATCH"],
            step_size=336,                       # DENSE rolling origin
            random_seed=seed,
            # futr_exog_list=["total_mean","total_variance"],
            inference_windows_batch_size=1,
            accelerator="auto"
        )
        nf = NeuralForecast(models=[model], freq=FREQ)
        nf.fit(df=df_nf[["unique_id","ds","y","total_mean","total_variance"]])

        Y_hat = nf.cross_validation(
            df=df_nf[["unique_id","ds","y","total_mean","total_variance"]],
            val_size=VAL_SIZE,
            test_size=TEST_SIZE,
            n_windows=None,   # as many as possible
            step_size=1,
            verbose=False
        ).reset_index()

        # Column with model output
        ycol = "NHITS" if "NHITS" in Y_hat.columns else [c for c in Y_hat.columns if c.upper()=="NHITS"][0]

        if "unique_id" not in Y_hat.columns:
            Y_hat["unique_id"] = df_nf["unique_id"].iloc[0] if len(df_nf["unique_id"].unique())==1 else "series_0"

        # Deduplicate any overlapping rows; keep last occurrence
        before = len(Y_hat)
        Y_hat = Y_hat.drop_duplicates(subset=["unique_id","ds"], keep="last")
        print(f"[CV] seed {seed}: forecasts={len(Y_hat)} (dedup {before - len(Y_hat)})")

        # Keep only merge columns and rename forecast
        yhat_seed_col = f"yhat_seed{seed}"
        Y_hat = Y_hat[["unique_id","ds", ycol]].rename(columns={ycol: yhat_seed_col})

        # Left-merge onto the dense test grid so every test timestamp is present
        filled = dense_test.merge(Y_hat, on=["unique_id","ds"], how="left")
        coverage = filled[yhat_seed_col].notna().mean()
        print(f"[CV] seed {seed}: coverage on test grid = {coverage*100:.2f}%")

        per_seed_frames.append(filled[["unique_id","ds", yhat_seed_col]])

    # 8) Combine all seeds into a single wide table + keep y once
    out = dense_test.copy()
    for f in per_seed_frames:
        out = out.merge(f, on=["unique_id","ds"], how="left")

    # Warn if some test timestamps still lack forecasts
    seed_cols = [c for c in out.columns if c.startswith("yhat_seed")]
    has_any = out[seed_cols].notna().any(axis=1)
    missing = (~has_any).sum()
    if missing:
        print(f"[WARN] {missing} test rows have no forecasts from any seed (often first few steps if HORIZON>1).")

    save_cv = os.path.join(cfg["OUT_DIR"], "cv_forecasts_all_seeds.csv")
    out.to_csv(save_cv, index=False)
    print(f"\n[CV] Saved dense backcast file → {save_cv}")

    # 9) Optional quick plot on the last seed’s median (if present)
    if cfg["MAKE_QUICK_PLOT"]:
        try:
            import matplotlib.pyplot as plt
            plot_df = out.copy().sort_values("ds")
            last_seed = f"yhat_seed{cfg['SEEDS_FOR_CV'][-1]}"
            plt.figure(figsize=(10,4))
            plt.plot(plot_df["ds"], plot_df["y"], label="Actual", linewidth=1)
            if last_seed in plot_df.columns:
                plt.plot(plot_df["ds"], plot_df[last_seed], label=f"Forecast ({last_seed})", linewidth=2)
            plt.title("Test backcasts (dense rolling origin)")
            plt.xlabel("ds"); plt.ylabel("y"); plt.legend(); plt.tight_layout()
            png_path = os.path.join(cfg["OUT_DIR"], "quick_cv_test_plot.png")
            plt.savefig(png_path, dpi=120); plt.close()
            print(f"[PLOT] Saved → {png_path}")
        except Exception as e:
            print(f"(Plot skipped) {e}")

    print("[DONE] All artifacts saved.")

# Run
main(CONFIG)

In [ ]:
# --- Block 3: Forecast Accuracy (MAPE Summary from 5-seed CV) ---

import os
import numpy as np
import pandas as pd
from scipy import stats

# ==== paths ====
OUT_DIR = "/content/drive/MyDrive/myproject/results_mdn_nhits"  # <-- match your model block
COMBINED_PATH = os.path.join(OUT_DIR, "cv_forecasts_all_seeds.csv")
SUMMARY_OUT   = os.path.join(OUT_DIR, "mape_summary.csv")
DAILY_OUT     = os.path.join(OUT_DIR, "mape_daily_detail.csv")

# ==== load ====
df = pd.read_csv(COMBINED_PATH)
if "ds" not in df.columns:
    raise ValueError(f"{COMBINED_PATH} must have a 'ds' column.")
df["ds"] = pd.to_datetime(df["ds"], utc=False)

# if unique_id wasn't persisted, create one (single-series case)
if "unique_id" not in df.columns:
    df["unique_id"] = "series_0"

# drop exact duplicate timestamps (keep last by default)
before = len(df)
df = df.drop_duplicates(subset=["unique_id", "ds"])
print(f"[DEDUP] Removed {before - len(df)} duplicate rows before MAPE.")

# identify the forecast columns produced by the model block
seed_cols = [c for c in df.columns if c.startswith("yhat_seed")]
if not seed_cols:
    raise ValueError("No yhat_seedN columns found. Did you run the model block that saves cv_forecasts_all_seeds.csv?")

need = {"unique_id","ds","y"}
missing = need - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns in forecast file: {missing}")

# ---- compute daily actuals & per-seed daily forecasts ----
df["date"] = df["ds"].dt.date

# Build a daily frame with actuals once
daily_actual = (
    df.groupby(["unique_id","date"], as_index=False)["y"]
      .sum()
      .rename(columns={"y":"y_daily"})
)

# For each seed, compute daily sums and daily MAPE, then collect
daily_list = []
for col in seed_cols:
    # some CV rows may be NaN (no forecast that day/hour). Keep only rows with predictions
    part = df[["unique_id","date",col]].dropna(subset=[col]).copy()
    if part.empty:
        # No forecasts for this seed; skip
        continue

    daily_seed = (
        part.groupby(["unique_id","date"], as_index=False)[col]
            .sum()
            .rename(columns={col: "yhat_daily"})
    )
    merged = daily_actual.merge(daily_seed, on=["unique_id","date"], how="inner")
    if merged.empty:
        continue

    # daily MAPE for this seed (ignore zero-actual days)
    merged["MAPE"] = np.where(
        merged["y_daily"] != 0.0,
        100.0 * np.abs(merged["y_daily"] - merged["yhat_daily"]) / merged["y_daily"],
        np.nan
    )
    merged["seed"] = col  # tag which seed
    daily_list.append(merged[["unique_id","date","seed","y_daily","yhat_daily","MAPE"]])

if not daily_list:
    raise ValueError("No overlapping daily actual/forecast values to evaluate.")

daily_all = pd.concat(daily_list, ignore_index=True)

# save detailed per-day-per-seed results
daily_all.to_csv(DAILY_OUT, index=False)
print(f"[DETAIL] Saved per-day per-seed MAPE → {DAILY_OUT}")

# ---- reduce to one MAPE per seed (mean across its days), then summarize across seeds ----
summary_rows = []
for uid, g in daily_all.groupby("unique_id"):
    # mean MAPE per seed (across that seed's evaluated days)
    per_seed_means = (
        g.groupby("seed", as_index=False)["MAPE"]
         .mean(numeric_only=True)
         .dropna(subset=["MAPE"])
    )
    mape_vals = per_seed_means["MAPE"].to_numpy()
    n = len(mape_vals)

    mean_mape = float(np.mean(mape_vals)) if n else np.nan
    if n > 1:
        std_mape = float(np.std(mape_vals, ddof=1))
        se = std_mape / np.sqrt(n)
        t_crit = stats.t.ppf(0.975, df=n-1)
        margin = float(t_crit * se)
        ci_expr = f"{mean_mape:.2f} ± {margin:.2f}"
    else:
        margin = np.nan
        ci_expr = "NA"

    summary_rows.append({
        "unique_id": uid,
        "Seeds_Used": n,
        "Mean_MAPE": mean_mape,
        "Margin_of_Error": margin,
        "CI_95": ci_expr
    })

summary_df = pd.DataFrame(summary_rows).sort_values("unique_id")
print("\n--- Mean of per-seed MAPEs (±95% CI across seeds) ---")
print(summary_df.to_string(index=False))

summary_df.to_csv(SUMMARY_OUT, index=False)
print(f"\n[SUMMARY] Saved → {SUMMARY_OUT}")